# naive_with_rc

In [3]:
def readGenome(filename):
    genome = ''
    with open(filename, 'r') as f:
        for line in f:
            # ignore header line with genome information
            if not line[0] == '>':
                genome += line.rstrip()
    return genome

def readFastq(filename):
    sequences = []
    qualities = []
    with open(filename) as fh:
        while True:
            fh.readline()  # skip name line
            seq = fh.readline().rstrip()  # read base sequence
            fh.readline()  # skip placeholder line
            qual = fh.readline().rstrip() # base quality line
            if len(seq) == 0:
                break
            sequences.append(seq)
            qualities.append(qual)
    return sequences, qualities

def reverseComplement(s):
    complement = {'A': 'T', 'C': 'G', 'G': 'C', 'T': 'A', 'N': 'N'}
    t = ''
    for base in s:
        t = complement[base] + t
    return t

def naive(p, t):
    occurrences = []
    for i in range(len(t) - len(p) + 1):  # loop over alignments
        match = True
        for j in range(len(p)):  # loop over characters
            if t[i+j] != p[j]:  # compare characters
                match = False
                break
        if match:
            occurrences.append(i)  # all chars matched; record
    return occurrences

In [4]:
def match(pattern, genome, pos):
    for i in xrange(len(pattern)):
        if pattern[i] != genome[pos + i]:
            return False
    return True

def naive_with_rc(pattern, genome):
    occurrences = []
    complementary = reverseComplement(pattern)
    for pos in xrange(len(genome) - len(pattern) + 1):
        if match(pattern, genome, pos) or match(complementary, genome, pos):
            occurrences.append(pos)            
    return occurrences

### Example 1

In [5]:
p = 'CCC'
ten_as = 'AAAAAAAAAA'
t = ten_as + 'CCC' + ten_as + 'GGG' + ten_as
occurrences = naive_with_rc(p, t)
print(occurrences) # should be [10, 23]

[10, 23]


### Example 2

In [6]:
p = 'CGCG'
t = ten_as + 'CGCG' + ten_as + 'CGCG' + ten_as
occurrences = naive_with_rc(p, t)
print(occurrences) # should be [10, 24]

[10, 24]


### Example 3

In [7]:
# Phi-X genome
!wget http://d396qusza40orc.cloudfront.net/ads1/data/phix.fa

--2018-07-24 23:17:29--  http://d396qusza40orc.cloudfront.net/ads1/data/phix.fa
Resolving d396qusza40orc.cloudfront.net (d396qusza40orc.cloudfront.net)... 13.35.23.59, 13.35.23.80, 13.35.23.75, ...
Connecting to d396qusza40orc.cloudfront.net (d396qusza40orc.cloudfront.net)|13.35.23.59|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5528 (5.4K) [application/octet-stream]
Saving to: <<phix.fa>>

phix.fa             100%[===================>]   5.40K  --.-KB/s    in 0s      

2018-07-24 23:17:29 (20.4 MB/s) - <<phix.fa>> saved [5528/5528]



In [8]:
phix_genome = readGenome('phix.fa')
occurrences = naive_with_rc('ATTA', phix_genome)
print('offset of leftmost occurrence: %d' % min(occurrences)) # should be 62

offset of leftmost occurrence: 62


In [9]:
print('# occurrences: %d' % len(occurrences)) # should be 60

# occurrences: 60


# lambda virus genome

In [10]:
!wget https://d28rh4a8wq0iu5.cloudfront.net/ads1/data/lambda_virus.fa

--2018-07-24 23:17:36--  https://d28rh4a8wq0iu5.cloudfront.net/ads1/data/lambda_virus.fa
Resolving d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)... 13.35.23.143, 13.35.23.216, 13.35.23.49, ...
Connecting to d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)|13.35.23.143|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 49270 (48K) [application/octet-stream]
Saving to: <<lambda_virus.fa>>

lambda_virus.fa     100%[===================>]  48.12K  --.-KB/s    in 0.04s   

2018-07-24 23:17:37 (1.07 MB/s) - <<lambda_virus.fa>> saved [49270/49270]



In [11]:
lambdaGenome = readGenome('lambda_virus.fa')

In [12]:
print len(naive_with_rc('AGGT', lambdaGenome))

306


In [13]:
print len(naive_with_rc('TTAA', lambdaGenome))

195


In [14]:
print naive_with_rc('ACTAAGT', lambdaGenome)[0]

26028


In [15]:
print naive_with_rc('AGTCGA', lambdaGenome)[0]

450


In [16]:
def naive_2mm(p, t):
    occurrences = []
    for i in range(len(t) - len(p) + 1):
        mm = 0
        for j in range(len(p)):
            if t[i+j] != p[j]:
                mm += 1
                if mm > 2:
                    break
        if mm <= 2:
            occurrences.append(i)
    return occurrences

print naive_2mm('ACTTTA', 'ACTTACTTGATAAAGT') # should be [0, 4]

[0, 4]


In [17]:
print len(naive_2mm('TTCAAGCC', lambdaGenome))

191


In [18]:
print naive_2mm('AGGAGGTT', lambdaGenome)[0]

49


# poor quality genome

In [19]:
!wget https://d28rh4a8wq0iu5.cloudfront.net/ads1/data/ERR037900_1.first1000.fastq



--2018-07-24 23:17:51--  https://d28rh4a8wq0iu5.cloudfront.net/ads1/data/ERR037900_1.first1000.fastq
Resolving d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)... 13.35.23.143, 13.35.23.216, 13.35.23.49, ...
Connecting to d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)|13.35.23.143|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 241626 (236K) [application/octet-stream]
Saving to: <<ERR037900_1.first1000.fastq>>

ERR037900_1.first10 100%[===================>] 235.96K  --.-KB/s    in 0.1s    

2018-07-24 23:17:51 (1.59 MB/s) - <<ERR037900_1.first1000.fastq>> saved [241626/241626]



In [20]:
reads, qualities = readFastq('ERR037900_1.first1000.fastq')

In [21]:
print naive('N', reads[1])
print naive('N', reads[99])
print naive('N', reads[199])


[66, 91]
[66]
[66]
